<a href="https://colab.research.google.com/github/goodofd3ath2/AspNetCore.Docs/blob/master/C%C3%B3pia_de_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import calendar
import pandas as pd
import plotly.express as px
from pathlib import Path
from IPython.display import display
import plotly.io as pio
import numpy as np
from collections import Counter

pio.templates.default = "ggplot2"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files 
uploaded = files.upload() #entro com os dados a serem trabalhados

Saving friends_info.csv to friends_info.csv


In [6]:
path = 'friends_info.csv' #crio o caminho

In [49]:
df = pd.read_csv(path) #chamo o caminho 
print(df.shape) #mostro na tela
display(df.sample(10)) #escolho quantas linhas vão ser mostradas na tela

(229, 10)


,Date,Episode,Title,Directed by,Written by,Duration,Summary,Rating/Share,U.S. viewers,Prod.\ncode
152,01/11/01,07-12,The One Where They're Up All Night,Kevin S. Bright,Zachary Rosenblatt,22.0,After the gang head up to the roof to see a pa...,14.8/23,22.86 million,226413
12,01/19/95,01-13,The One with the Boobies,Alan Myerson,Alexa Junge,22.0,Joey finds out his dad has a mistress. Rachel ...,17.3/26,25.8 million,456664
108,02/11/99,05-14,The One Where Everybody Finds Out,Michael Lembeck,Alexa Junge,22.0,When Phoebe finds out about Monica and Chandle...,17.8/28,27.7 million,467664
79,11/20/97,04-08,The One with Chandler in a Box,Peter Bonerz,Michael Borkow,22.0,Chandler tries to earn Joey's forgiveness by l...,18.4/29,26.8 million,466608
160,04/19/01,07-19,The One with Ross and Monica's Cousin,Gary Halvorson,Andrew Reich & Ted Cohen,22.0,Joey auditions for a naked role in a big movie...,11.4/19,16.55 million,226421
71,05/15/97,03-25,The One at the Beach,Pamela Fryman,Story by : Pang-Ni Landrum & Mark Kunerth\nTel...,22.0,"While at the beach, Phoebe meets a family frie...",16.5/29,28.8 million,465275
154,02/08/01,07-14,The One Where They All Turn Thirty,Ben Weiss,Story by : Vanessa McCarthy\nTeleplay by : Ell...,22.0,Rachel's 30th birthday prompts the rest of the...,13.9/21,22.40 million,226411
169,10/25/01,08-05,The One with Rachel's Date,Gary Halvorson,Brian Buckner & Sebastian Jones,22.0,Phoebe dates Monica's sous-chef Tim and Monica...,16.1/25,25.64 million,227404
149,12/07/00,07-09,The One with All the Candy,David Schwimmer,Wil Calhoun,22.0,Monica makes candy to get to know her neighbou...,13.6/22,21.08 million,226408
217,11/13/03,10-07,The One with the Home Study,Kevin S. Bright,Mark Kunerth,22.0,Monica and Chandler nervously prepare for a vi...,13.2/21,20.21 million,176257


In [57]:
# Clean Episodes column
df['Season-Episode'] = df['Episode']
cleaned_column = []
for i, val in enumerate(df.Episode):
    if '\n' in val:
        val1, val2 = val.split('\n')
        val = f"{val1}{val2.split('-')[1]}"
    cleaned_column.append(val)
df.Episode = cleaned_column

# Separate Month, Day, and Year from Date column
timedf = pd.DataFrame(
    data=list(map(lambda x: list(map(int, x.split("/"))), df.Date)), 
    columns=["Month", "Day", "Year"]
)
# df.Date = pd.to_datetime(popdf.Date) # convert string to date format

# Make new columns with season number and episode number
df.Episode = df.Episode.replace({"Special": "10-100"}) # Some cleaning
sedf = pd.DataFrame(list(map(lambda x: x.split("-"), df.Episode)), columns=["Season", "Episode"])
df = df.drop('Episode', axis=1)
df = pd.concat((timedf, df, sedf), axis=1)

# Remove million from the count
df["U.S. viewers"] = df["U.S. viewers"].apply(lambda x: float(x.replace(' million', '')))
df = df.rename(columns={"U.S. viewers": "U.S. viewers (in million)"})

In [59]:
df[df['U.S. viewers (in million)'] == df['U.S. viewers (in million)'].max()].T

,35
Month,1
Day,28
Year,96
Date,01/28/96
Title,The One After the Super Bowl
Directed by,Michael Lembeck
Written by,Jeffrey Astrof & Mike SikowitzMichael Borkow
Duration,44
Summary,Ross goes to visit Marcel whilst on a trip to ...
Rating/Share,29.6/46


In [21]:
len(df.index) #mostra quantas linhas tem

229

In [62]:
px.line(df, x='Date', y='U.S. viewers (in million)', title='U.S. Viewers trend over years',
        hover_data=['Title', 'Season-Episode'])

In [63]:
df["Rating"] = df["Rating/Share"].apply(lambda x: float(x.split("/")[0]))
px.line(df, y="Rating", x="Date", title='Rating trend over years', hover_data=['Title', 'Season-Episode'])

In [64]:
df["Share"] = df["Rating/Share"].apply(lambda x: float(x.split("/")[1]))
px.line(df, y="Share", x="Date", title='Share trend over years', hover_data=['Title', 'Season-Episode'])

In [65]:
epdf = df.copy().dropna().reset_index(drop=True)
px.scatter(epdf.dropna(), x='Episode', y='Duration',size="Duration", color='Duration',
           color_continuous_scale=px.colors.sequential.RdBu, hover_data=['Title', 'Season-Episode'])